In [ ]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image
from matplotlib.pyplot import imshow

In [ ]:
class DoodleCNN():
    def __init__(self, data_dir, limit=None, split=0.05):
        self.classes = {}
        self.load(data_dir, split, limit)
        self.buildModel()
        
    def load(self, data_dir, split=0.05, limit=None):
        X = []
        Y = []
        label = -1
        for file in os.listdir(data_dir):
            if file.endswith(".npy"):
                label += 1
                contents = self.reshape(np.load(data_dir + file)[0:limit])
                self.classes[label] = file[:-4]
                X = X + contents
                Y = Y + [label for _ in range(0, len(contents))]
        X = self.normalize(X)
        self.X_train, self.X_test, self.Y_train, self.Y_test = self.split(X, Y, split)
        
    def normalize(self, data):
        return np.interp(data, [0, 255], [-1, 1])
            
    def split(self, X, Y, split=0.05):
        X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=split)
        Y_train = np_utils.to_categorical(Y_train, len(self.classes))
        Y_test = np_utils.to_categorical(Y_test, len(self.classes))
        return X_train, X_test, Y_train, Y_test
    
    def buildModel(self):
        self.model = Sequential()
        self.model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(len(self.classes), activation='softmax'))
        
    def train(self, epochs=10):
        self.model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
        self.model.fit(np.array(self.X_train), np.array(self.Y_train), batch_size=32, epochs=epochs, shuffle=True)
                
    def reshape(self, contents):
        reshaped = []
        for i in range(len(contents)):
            image = np.reshape(contents[i], (28, 28, 1))
            reshaped.append(image)
        return reshaped
        

In [ ]:
nn = DoodleCNN("data/", 5000, 0.05)
print(nn.classes)
print(np.shape(nn.X_train), np.shape(nn.Y_train), np.shape(nn.X_test), np.shape(nn.Y_test))
print(len(nn.classes))

In [ ]:
nn.model.summary()

In [ ]:
nn.train()